In [4]:
# Importando las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Importando las librerías para la selección de características
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import optuna

/Users/cash/Desktop/F5/Python/Machine_Learning/non-supervised-learning/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Cargar los datos de entrenamiento y prueba
data = pd.read_csv('train.csv')

In [6]:
data.shape

(8693, 14)

In [7]:
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [19]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# ======== Ingeniería de características ========

# 1. Grupos de edad
def categorize_age(age):
    if pd.isna(age):
        return 'Unknown'
    elif age < 18:
        return 'Child'
    elif age < 30:
        return 'Young Adult'
    elif age < 60:
        return 'Adult'
    else:
        return 'Senior'
    
data['AgeGroup'] = data['Age'].apply(categorize_age)

# 2. PassengerId -> group y group_size
data['PassengerId'] = data['PassengerId'].astype(str)
data['Group'] = data['PassengerId'].apply(lambda x: x.split('_')[0])
data['Group_size'] = data.groupby('Group')['Group'].transform('count')

# 3. Cabin -> Deck, Number, Side
def split_cabin(cabin):
    if isinstance(cabin, str):  # Solo procesar si es una cadena
        parts = cabin.split('/')
        deck = parts[0] if len(parts) > 0 else 'Unknown'
        number = parts[1] if len(parts) > 1 else 'Unknown'
        side = parts[2] if len(parts) > 2 else 'Unknown'
        return pd.Series([deck, number, side])
    else:
        return pd.Series(['Unknown', 'Unknown', 'Unknown'])

data[['CabinDeck', 'CabinNumber', 'CabinSide']] = data['Cabin'].apply(split_cabin)

# 4. Name -> Surname
data['Surname'] = data['Name'].apply(lambda x: x.split()[-1] if isinstance(x, str) else 'Unknown')


# ======== Imputación avanzada de valores faltantes ========

# Crear función para imputar usando Group
def impute_with_group(feature):
    data[feature] = data.groupby('Group')[feature].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))

# Imputar valores faltantes para HomePlanet
impute_with_group('HomePlanet')

# Si aún hay valores faltantes en HomePlanet, usar CabinDeck
data['HomePlanet'] = data.groupby('CabinDeck')['HomePlanet'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))

# Imputar valores faltantes para Destination usando HomePlanet
data['Destination'] = data.groupby('HomePlanet')['Destination'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))

# Imputar valores faltantes para CryoSleep y VIP usando Group y Surname
impute_with_group('CryoSleep')
impute_with_group('VIP')

# Imputar valores faltantes de CabinSize usando Group o Surname
data['CabinSize'] = data.groupby('Surname')['CabinNumber'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))

# Para otras características, simplemente utilizamos imputación media o más lógica
imputer = SimpleImputer(strategy='most_frequent')
data[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = imputer.fit_transform(data[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])

# ======== Codificación de variables categóricas ========
continuous_cols = data.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = data.select_dtypes(include=['object']).columns

# Codificar las variables categóricas utilizando OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_features = encoder.fit_transform(data[categorical_cols])

# Añadir estas nuevas características al dataset
df_encoded = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))
data = pd.concat([data, df_encoded], axis=1)

# Eliminar columnas originales innecesarias
data = data.drop(columns=categorical_cols.tolist() + ['PassengerId', 'Cabin', 'Name'])

# ======== Entrenamiento del modelo ========

# Dividir los datos en train y test
X = data.drop(columns=['Transported'])  # Features
y = data['Transported']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Hacer predicciones y evaluar el modelo
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.7826


In [9]:
# from sklearn.impute import SimpleImputer

# # Identificar características continuas y categóricas
# continuous_cols = data.select_dtypes(include=['float64', 'int64']).columns
# categorical_cols = data.select_dtypes(include=['object']).columns

# # Imputación de la mediana para las características continuas
# imputer_median = SimpleImputer(strategy='median')
# data[continuous_cols] = imputer_median.fit_transform(data[continuous_cols])

# # Imputación de la moda para las características categóricas
# imputer_mode = SimpleImputer(strategy='most_frequent')
# data[categorical_cols] = imputer_mode.fit_transform(data[categorical_cols])

# # Eliminar la columna PassengerId ya que es un identificador único
# # data.drop('PassengerId', inplace=True, axis=1)

# # Convertir las columnas mixtas en tipo string (si aplica)
# for col in data.columns:
#     if data[col].dtype == 'object':
#         data[col] = data[col].astype(str)

# # Aplicar Label Encoding a las columnas categóricas
# oe = {}
# for col in categorical_cols:
#     oe[col] = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
#     data[col] = oe[col].fit_transform(data[col].values.reshape(-1, 1))

# # Separar características (X) y la variable objetivo (y)
# X = data.drop('Transported', axis=1)
# y = data['Transported']

# # Dividir los datos en conjuntos de entrenamiento y prueba
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

(((6954, 17406), (6954,)), ((1739, 17406), (1739,)))

In [23]:
# Definir la función de objetivo
def objective(trial):
    # Hiperparámetros a optimizar
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    # Entrenar el modelo
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=42)
    clf.fit(X_train, y_train)

    # Hacer predicciones y evaluar el modelo
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

# Ejecutar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Mejor combinación de hiperparámetros
best_params = study.best_params
print(f'Best hyperparameters: {best_params}')

# Entrenar el modelo con los mejores hiperparámetros
best_clf = RandomForestClassifier(**best_params, random_state=42)
best_clf.fit(X_train, y_train)

# Evaluar el modelo optimizado
y_pred_best = best_clf.predict(X_test)
best_accuracy = accuracy_score(y_test, y_pred_best)
print(f'Best Accuracy: {best_accuracy:.4f}')


[I 2024-09-23 18:14:15,334] A new study created in memory with name: no-name-5013b34c-0fa5-4ba4-a6a9-c9dfaaf7271c
[I 2024-09-23 18:14:17,033] Trial 0 finished with value: 0.679700977573318 and parameters: {'n_estimators': 61, 'max_depth': 21, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.679700977573318.
[I 2024-09-23 18:14:18,962] Trial 1 finished with value: 0.6820011500862565 and parameters: {'n_estimators': 82, 'max_depth': 19, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.6820011500862565.
[I 2024-09-23 18:14:21,152] Trial 2 finished with value: 0.6762507188039103 and parameters: {'n_estimators': 179, 'max_depth': 19, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.6820011500862565.
[I 2024-09-23 18:14:23,057] Trial 3 finished with value: 0.679700977573318 and parameters: {'n_estimators': 130, 'max_depth': 8, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.6820011500862565.
[I 2024-09-23 18:14:24,808] Trial 4 finished with value: 0.6768257619321449 and

Best hyperparameters: {'n_estimators': 141, 'max_depth': 28, 'min_samples_leaf': 1}
Best Accuracy: 0.7476


In [21]:
def objective(trial):
    """
    Defining the objective function for Optuna with feature selection
    """
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    # Number of features to select
    k_best = trial.suggest_int('k_best', 1, X_train.shape[1])  

    # Feature selection
    select_k_best = SelectKBest(score_func=f_classif, k=k_best) # SelectKBest: Esta clase se utiliza para seleccionar las k mejores características según un criterio de puntuación. En este caso, f_classif se utiliza como la función de puntuación, que es adecuada para clasificación.
    preprocessor = ColumnTransformer([('select', select_k_best, X.columns)], remainder='passthrough') # Se define un transformador de columnas que aplica la selección de características a las columnas especificadas en X.columns. El parámetro remainder='passthrough' asegura que las columnas no seleccionadas se mantengan sin cambios.
    
    # Building the model pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42))]) # Se crea un pipeline que encadena el preprocesador y el clasificador. Esto simplifica el proceso de entrenamiento y predicción, asegurando que las transformaciones se apliquen correctamente. RandomForestClassifier: Se inicializa el clasificador con los hiperparámetros sugeridos (número de estimadores y profundidad máxima).

    clf.fit(X_train, y_train) # Se entrena el modelo (clf) usando los datos de entrenamiento (X_train, y_train).
    y_pred = clf.predict(X_test) # Se realizan predicciones sobre el conjunto de prueba (X_test).
    accuracy = accuracy_score(y_test, y_pred) # Se calcula la precisión del modelo utilizando la función accuracy_score, comparando las predicciones (y_pred) con las etiquetas verdaderas (y_test).
    
    # Storing the trained classifier in the study object
    trial.set_user_attr('classifier', clf) # almacena el clasificador entrenado en el objeto trial. Esto puede ser útil para análisis posteriores o para evaluar el rendimiento del modelo después de la optimización
    return accuracy

In [22]:
# Running the optimization
study = optuna.create_study(study_name='best_features_and_hyperparameters', direction='maximize') # optuna.create_study(...) es una función que se utiliza para crear un nuevo objeto de estudio. Un estudio en Optuna es un contenedor para todas las pruebas (trials) que se realizarán durante la optimización. Tiene dos parámetros: 1. study_name='best_features_and_hyperparameters': Este es el nombre que se le asigna al estudio. Puedes usar este nombre para identificar el estudio en la base de datos de Optuna, especialmente si estás guardando resultados o usando la interfaz de visualización. 2. direction='maximize': Este parámetro indica que el objetivo de la optimización es maximizar el valor que se devolverá en la función objetivo (en este caso, la precisión del modelo). Si en lugar de maximizar quisieras minimizar un valor (como la pérdida), usarías direction='minimize'.
study.optimize(objective, n_trials=10) # study.optimize(...): es un método que ejecuta el proceso de optimización, llamando repetidamente a la función objetivo para encontrar la mejor combinación de hiperparámetros y características. Tiene dos parámetros: 1. objective: Este es el nombre de la función objetivo que se definió anteriormente. Optuna la llamará múltiples veces con diferentes configuraciones de hiperparámetros, que se generan automáticamente. 2. n_trials=50: Este parámetro indica el número total de pruebas (trials) que se realizarán. En cada prueba, Optuna selecciona un conjunto diferente de hiperparámetros y características que serán probados en el modelo. En este caso, se realizarán 50 pruebas diferentes.

[I 2024-09-23 18:10:33,174] A new study created in memory with name: best_features_and_hyperparameters
/Users/cash/Desktop/F5/Python/Machine_Learning/non-supervised-learning/env/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [   10    11    14 ... 17403 17404 17405] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Users/cash/Desktop/F5/Python/Machine_Learning/non-supervised-learning/env/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
[I 2024-09-23 18:10:43,975] Trial 0 finished with value: 0.7492811960897068 and parameters: {'n_estimators': 287, 'max_depth': 6, 'k_best': 8751}. Best is trial 0 with value: 0.7492811960897068.
/Users/cash/Desktop/F5/Python/Machine_Learning/non-supervised-learning/env/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py

KeyboardInterrupt: 

In [24]:
# Extracting the best hyperparameters and selected features
print('Best hyperparameters: ', study.best_params) # study.best_params: Este atributo contiene un diccionario con los mejores hiperparámetros que Optuna encontró durante el proceso de optimización. Los hiperparámetros se refieren a los valores óptimos que maximizan el rendimiento del modelo según la función objetivo definida.
best_trial = study.best_trial # study.best_trial: Este atributo proporciona acceso al trial (prueba) que produjo el mejor resultado durante la optimización. Incluye información sobre los hiperparámetros utilizados, el valor de la función objetivo (por ejemplo, la precisión del modelo), y otros metadatos relevantes.
selected_features = SelectKBest(score_func=f_classif, 
                                k=best_trial.params['k_best']).fit(X_train, y_train) # SelectKBest: Esta clase se utiliza para seleccionar las k mejores características según una función de puntuación. En este caso, f_classif es la función de puntuación que evalúa la relación entre las características y la variable objetivo.

                                # SelectKBest: Esta clase se utiliza para seleccionar las k mejores características según una función de puntuación. En este caso, f_classif es la función de puntuación que evalúa la relación entre las características y la variable objetivo.
                                
                                # k=best_trial.params['k_best']: Aquí, se utiliza el valor de k_best que se encontró durante la optimización como el número de características a seleccionar. Este valor se extrae del mejor trial.
                                
                                # .fit(X_train, y_train): Este método ajusta el selector de características a los datos de entrenamiento, calculando las puntuaciones de las características y seleccionando las mejores.

print('Selected features: ', X.columns[selected_features.get_support()]) # selected_features.get_support(): Este método devuelve un array booleano que indica qué características han sido seleccionadas por el SelectKBest. Devuelve True para las características seleccionadas y False para las que no lo son.

# X.columns[...]: Aquí se utilizan los índices booleanos para filtrar las columnas de X. Esto devuelve solo las columnas (características) que han sido seleccionadas.

Best hyperparameters:  {'n_estimators': 623, 'max_depth': 12, 'k_best': 12}
Selected features:  Index(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name'],
      dtype='object')


In [25]:
# Load Submission Data
submission_data = pd.read_csv('test.csv')

# Drop Rows with Missing Values
submission_data.fillna(0, inplace=True)

# Applying the label encoders to the submission data
for column, encoder in oe.items():
    submission_data[column] = encoder.transform(submission_data[column].values.reshape(-1, 1))
    
# Generate Sample Submission
best_clf = best_trial.user_attrs['classifier']
y_pred = best_clf.predict(submission_data)
submission_data['Transported'] = y_pred

# Only keep required columns for submission
submission_data = submission_data[['PassengerId', 'Transported']]

# Save the DataFrame to Kaggle Directory and Upload to Submit
submission_data.to_csv('submission.csv', index=False, header=True)